In [7]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.data import rollout
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
from imitation.data.types import Trajectory,DictObs,TrajectoryWithRew,Tuple
import h5py
from GailNavigationNetwork.model import NaviNet
from GailNavigationNetwork.utilities import preprocess




In [8]:
def create_demos(file_path,DEVICE="cuda"):
    '''
    Creates a gymnasium transition from the given file path
    of hdf5 file of known structure

    Args:
    file_path: str  
    Path to the hdf5 file   

    Returns:
    rollouts: gymnasium.Transition

    '''    
    read_file= h5py.File(file_path, "r")
    model= NaviNet().to(DEVICE)
    model.eval()
    len= read_file['kris_dynamics']['odom_data']['target_vector'].shape[0]
    rgbs=[]
    depths=[]
    targets=[]  
    acts=[]
    ob=[]
    for i in range(len):
        target=read_file['kris_dynamics']['odom_data']['target_vector'][i]
        rgb=read_file['images']['rgb_data'][i]
        depth=read_file['images']['depth_data'][i]
        act=read_file['kris_dynamics']['odom_data']['odom_data_wheel'][i]
        # print(f"depth shape in rollout {depth.shape}")
        rgb=preprocess(rgb)
        depth=preprocess(depth)
        # rgb,depth=preprocess(rgb,depth)
        (rgb, depth) = (rgb.to(DEVICE), depth.to(DEVICE))
        rgb_features, depth_features = model(rgb,depth)
        rgb_features=rgb_features.detach().cpu().numpy()
        depth_features=depth_features.detach().cpu().numpy()
        # rgbs.append(rgb_features)
        # depths.append(depth_features)
        # targets.append(target) 
        ob.append((target,rgb_features,depth_features))
        acts.append(act)
        

    acts=np.array(acts[:-1])
    dones=[False for i in range(len)]
    dones[-1]=True
    infos= [{} for i in range(len-1)]
    # rgbs=np.array(rgbs).reshape(25, 1, 1280, 8, 10)
    # depths=np.array(depths).reshape(25, 1,  238, 318)
    # targets=np.array(targets).reshape(25, 1, 7)


    # obs_tuple = (
    # targets.reshape(25, 1, 7),
    # rgbs.reshape(25, 1, 1280, 8, 10),
    # depths.reshape(25, 1,  238, 318)
    # )   


# Combine into one tuple

    # obs_dict=tuple((targets,
    #         rgbs,
    #          depths))
    # for i in range(len-1):
    #     obs_dict[0][i]=np.array(obs_dict[0][i])
    #     obs_dict[1][i]=np.array(obs_dict[1][i])
    #     obs_dict[2][i]=np.array(obs_dict[2][i])
    
    # print(f"obs_dict shape {obs_tuple[0].shape} {obs_tuple[1].shape} {obs_tuple[2].shape}")
    ob=np.array(ob)

    traj = Trajectory(obs=ob, acts=acts,infos=infos,terminal=dones)


    return rollout.flatten_trajectories([traj])



In [9]:
file_path="/home/foxy_user/foxy_ws/src/gail_navigation/GailNavigationNetwork/data/traj2.hdf5"
demonstrations,rollout=create_demos(file_path)
print(f"rollout {rollout[0]}")

AssertionError: 